# Lab 4 Exploration: Object Detection

In this notebook, we will learn how to use the racecar camera to identify objects based on their color and extract the center of these objects.  Next week, we will work toward commanding our robot to move with respect to those objects.

Throughout this notebook, **<font style="color:red">text in bold red</font>** indicates a change you must make to the following code block before running it.  **Do not change any other code.**

For this lab, run the code one block at a time using ctrl + enter (the control key and the enter key at the same time).  You will want to examine the output of the blocks of code.

## 1. Getting Started

First, we will set the file paths, import the Racecar interface, import the necessary libraries for this notebook (`cv`, `numpy`, etc.), and create a Racecar instance.  (Remember, only run this cell once.)  Wait for it to print `racecar created successfully`.

In [ ]:
# We will not be using simulation here, so this variable should never change
isSimulation = False

# Import Python libraries
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from nptyping import NDArray
from typing import Any, Tuple, List, Optional

# Import Racecar library
import sys
sys.path.append(1, "../../library")
import racecar_core
import racecar_utils as rc_utils

# Create Racecar
rc = racecar_core.create_racecar(isSimulation)

Next we will add some helper functions that will be useful for this lab. Run this cell only once. When it is done it should print `helpers added successfully`. 

In [ ]:
def show_color_bgr(blue, green, red):
    """
    Displays a color specified in the BGR format.
    """
    rectangle = plt.Rectangle((0,0), 50, 50, fc=(red/255, green/255, blue/255))
    plt.gca().add_patch(rectangle)
    plt.show()

def show_color_hsv(hue, saturation, value):
    """
    Displays a color specified in the HSV format
    """
    # Convert from hsv to rgb
    hsv = np.array([[[hue, saturation, value]]], np.uint8)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    
    show_color_bgr(bgr[0][0][0], bgr[0][0][1], bgr[0][0][2])

def get_mask(image, hsv_lower, hsv_upper):
    """   
    Returns a mask containing all of the areas of image which were between hsv_lower and hsv_upper.
    
    Args:
        image: The image (stored in BGR) from which to create a mask.
        hsv_lower: The lower bound of HSV values to include in the mask.
        hsv_upper: The upper bound of HSV values to include in the mask.
    """
    # Convert hsv_lower and hsv_upper to numpy arrays so they can be used by OpenCV
    hsv_lower = np.array(hsv_lower)
    hsv_upper = np.array(hsv_upper)
    
    image = cv.cvtColor(image, cv.COLOR_RGB2HSV)
    mask = cv.inRange(image, hsv_lower, hsv_upper)
    
    return mask

print("helpers added successfully")

## 2. Navigating an image
Let's take a photo with the car's camera using `rc.camera.get_color_image()` and show it with `rc.display.show_color_image()`.  Both of these functions are specifically provided for the racecar.

**<font style="color:blue">Don't make any changes to the block of code.  Run it to take a picture.  Then change what is in front of the car and take a different picture, so you can see that these are live images being taken.</font>**

In [ ]:
# Initialize image with a deep copy of the most recent color image captured
# by the camera
image = rc.camera.get_color_image()

# Show the image captured by the camera
rc.display.show_color_image(image)

Use the coordinates on the output image above to estimate the dimensions of the image.
How many rows and columns are there?

The image was stored as an array with the name `image`.  You can get the `shape` (dimensions) of an array by writing the array's name, and then adding `.shape` to the end of the name.  Put those hints together to **<font style="color:red">write and run a line of code below that prints the exact dimensions of the image captured above </font>**.

In [ ]:
# TODO: write and run a line of code that prints the shape of the image array


What are the three numbers?

The color images we retrieve are stored as three dimensional arrays:

* **0th dimension**: pixel rows, indexed from top to bottom.
* **1st dimension**: pixel columns, indexed from left to right.
* **2nd dimension**: pixel color values, ordered blue, green, red, each ranging from 0 (none of that color) to 255 (maximum amount of that color).

Let's look at the color values of the middle pixel of our image.

**<font style="color:blue">Don't make any changes to the block of code. Predict what color the middle pixel will be. Then run the code to show the color of the middle pixel of the previous image.  Were you right?  If not, try centering a solid-colored object in front of the camera and take another picture.</font>**

In [ ]:
# Calculate center row and column
row = 240
col = 320

# Extract and print blue, green, and red values
blue = image[row][col][0]
green = image[row][col][1]
red = image[row][col][2]

print("blue:", blue)
print("green:", green)
print("red:", red)

# Display this color
show_color_bgr(blue, green, red)

# Also display the location of the center of the original image.
image_copy = image.copy()
rc_utils.draw_circle(image_copy, (row, col), rc_utils.ColorBGR.red.value)
rc.display.show_color_image(image_copy)

**<span style="color:blue">Place a solid-colored object (like your cone) near the edge of the image, but so the camera can still see it. Run the code block below to take a new picture.</span>**

In [ ]:
# Take image from camera and display it
image = rc.camera.get_color_image()
rc.display.show_color_image(image)

**<span style="color:red">Update `row` and `col` in the following code block to find a pixel that corresponds to your object, and get the RGB values of that pixel.  You can use the axes on the photo above to estimate good pixel coordinates.  What are the B, G, and R values for the object?  Do these B, G, and R values match what you might expect?</span>**

In [ ]:
#-------------------------------
# TODO: Identify the desired row and column
row = 0
col = 0
#-------------------------------


# Extract and print red, green, and blue values
blue = image[row][col][0]
green = image[row][col][1]
red = image[row][col][2]

print("blue:", blue)
print("green:", green)
print("red:", red)

# Display this color
show_color_bgr(blue, green, red)

# Also display the location of the pixel you selected on the original image.
image_copy = image.copy()
rc_utils.draw_circle(image_copy, (row, col), rc_utils.ColorBGR.red.value, radius=2)
rc.display.show_color_image(image_copy)

## 3. Color Formats
By default, the images captured by the camera are stored in the blue-green-red (BGR) color format.  However, when recognizing objects based on their color, it often easier to use the hue-saturation-value (HSV) format because it is more robust to differences in lighting and shading.  The channels in HSV correspond to the following:

* **Hue** (0 to 180): The color as it appears on a color wheel, ordered as red-orange-yellow-green-blue-purple-red
* **Saturation** (0 to 255): Related to the amount of white in the color. 0 is pure white, and 255 is the pure color without any white added.
* **Value** (0 to 255): Related to the amount of black in the color. 0 is pure black, and 255 is the pure color without any black added.

While saturation and value vary with lighting, hue will remain mostly the same regardless of lighting.  By focusing on the hue of the object we are attempting to detect, we can find it even in different lighting environments.

We can use the following widgets to experiment with different color values in the BGR and HSV formats.  You will need to run the code blocks to get the widgets to appear.

<b><font style="color:blue">For both formats, find the values which produce the following colors:
- green
- orange

You should have 4 sets of values at the end, one set for each of the 2 colors with each of the 2 color schemes.
</font></b>

Try to represent the same shade with both color schemes.  For consistency, you can try to match the provided green and orange cones.

In [ ]:
# BGR color
widgets.interact(
    show_color_bgr,
    blue=widgets.IntSlider(0, 0, 255, continuous_update=False),
    green=widgets.IntSlider(0, 0, 255, continuous_update=False),
    red=widgets.IntSlider(0, 0, 255, continuous_update=False),
);

In [ ]:
# HSV color
widgets.interact(
    show_color_hsv,
    hue=widgets.IntSlider(0, 0, 180, continuous_update=False),
    saturation=widgets.IntSlider(255, 0, 255, continuous_update=False),
    value=widgets.IntSlider(255, 0, 255, continuous_update=False),
);

## CHECKPOINT:  Write your HSV values on your board.  Talk to another team and compare your HSV values with their HSV values.  How are your answers similar or different? Do both teams' answers make sense?

## 4. Masks
Let's work on identifying an object in the car's field of view based on its color.  Specifically, we will isolate the portions of an image which fall within a certain color range by defining upper and lower HSV bounds. We will use that to create a *mask* - a special type of image which tells us which parts of an image to include and which parts to exclude.

When we apply a mask to an image, only some parts of the original image will show up in the output image.  (How does this compare to when a person wears a mask or partial mask?)

Let's isolate the colored cone(s) in an image.


<b><font style="color:blue">
Place one bright green cone in the car's line of sight and take a picture by running the following block of code.
</font></b>

If you do not have a green cone, you can use any object with a distinct color, but we recommend using the green cone first if you have one.  We recommend avoiding red as your first color. (Why?)

In [ ]:
image = rc.camera.get_color_image()
rc.display.show_color_image(image)

Next, we will use the `rc_utils.find_contours()` function to create a mask containing just the cone(s).  

We will choose which pixels to include or exclude from the mask based on the HSV values of the pixels.  If the HSV values lie between `hsv_lower` and `hsv_upper`, the pixels will be included.  We specify `hsv_lower` and `hsv_upper` as 3 numbers each, in the order H, S, V.

When you first receive this code, `hsv_lower` is (0, 0, 0) and `hsv_upper` is (180, 255, 255).  All possible HSV values lie between these values of `hsv_lower` and `hsv_upper`, so the mask will contain the entire image.  **<font style="color:red">Tune the values of `hsv_lower` and `hsv_upper` until the mask only includes the cone.  </font>**  


**Hints:**

* Use the HSV color widget from the Color Formats section above to visualize HSV colors.
* Saturation and value vary a lot with lighting, but hue will remain mostly constant for a given object. Try using a wide range for value and saturation but a tight range for hue.
* If you have both masked and unmasked areas, the area that will be included will appear yellow in the output image, and the part to be excluded will appear blue/purple.  (If the whole image is included, as you initially received it, the whole mask will appear blue/purple.)

In [ ]:
#-------------------------------
# TODO: change these bounds
hsv_lower = (0, 0, 0) # (hue, saturation, value)
hsv_upper = (180, 255, 255) # (hue, saturation, value)
#-------------------------------

# Given color bounds, create a mask
mask = get_mask(image, hsv_lower, hsv_upper)

# Display the mask
rc.display.show_color_image(mask)

We can use this mask as a filter for our original image to only keep portions that were between `hsv_lower` and `hsv_upper`.  The portions we include will appear in the new image as they did in the original image.  The portions that we exclude will appear black in the new image, instead of showing the content they used to contain.

<b><font style="color:blue">
Run the next cell and confirm that your colored cone still appears the correct color, while the background is replaced by solid black.  If it doesn't, tune your HSV values again to adjust the mask.
</font></b>


In [ ]:
masked_image = cv.bitwise_and(image, image, mask=mask)
rc.display.show_color_image(masked_image)

## CHECKPOINT:  Compare your values for `hsv_lower` and `hsv_upper` with another team's.  If your objects are the same color (e.g., both green cones), are your values similar?  If your objects are different colors (e.g., green cone and orange cone), do the other team's values make sense?

## 5. Finding Contours

Now that we have a mask, we can create outlines around each object in the mask. We will use these outlines to identify the largest object and calculates its size and position. These outlines are often referred to as *contours*, which is what we will be calling them from now on.

The following steps have been done for you in the code.

First, we use the function `rc_utils.find_contours()` to create a list of contours around each distinct object in the mask.  There may be more than one contour if there are multiple cones in the image or if there are other objects that have a similar color to the cone(s).  Next, we use `rc_utils.get_largest_contour()` to find the largest contour and draw it on the image.

<b><font style="color:blue">
Run the next cell and confirm that you see a green outline surrounding the closest cone in your image.  If you want, you can coordinate with another team to borrow their cone so you can confirm that only the larger contour is outlined when multiple contours are present.
</font></b>

Note: If you want to use a new image, you will need to re-run the code blocks starting from Section 4 to take an image, compute the mask, and then compute contours.


In [ ]:
# Find the largest contour
contours = rc_utils.find_contours(mask)
largest_contour = rc_utils.get_largest_contour(contours)

# Draw it on the image and display the updated image
image_copy = np.copy(image)
if (largest_contour is not None):
    rc_utils.draw_contour(image_copy, largest_contour)
    rc.display.show_color_image(image_copy)
else:
    print("No contours found")


One advantage of using contours is that it allows us to easily calculate the center of an object. We'll now calculate the center point of the cone, then draw a dot at that point.

<b><font style="color:blue">
Run the next cell and confirm that you see a small cyan (light blue) dot in the middle of the visible part of your cone.
</font></b>

In [ ]:
# Find the center of this contour if it exists
if (largest_contour is not None):
    center = rc_utils.get_contour_center(largest_contour)
    
    # Draw a circle at the contour center
    rc_utils.draw_circle(image_copy, center)
    rc.display.show_color_image(image_copy)
else:
    print("No contours found, so no center point to find")

## 6. Image Cropping

Often, the car will think it sees objects in the background that have the same color as the object you are trying to detect. This can be for many reasons. Here are a few:
- There is some colored lighting (e.g., caused by a sunset) that makes an object look a different color than it usually does.
- There's a reflective surface in the background, and the car is seeing a reflection of your object.

One way to avoid picking up on these extra detections is to only look at the ground in front of us while driving, instead of looking at the whole image. (For example, we might use this strategy if we are trying to follow a piece of tape on the ground, and don't need to look at the sky.)

**<font style="color:red">With your cone near the edge of the camera image, edit `top_left_corner` and `bottom_right_corner` until the image is roughly centered around the cone. </font>**

In [ ]:
# Take image from camera and display it
image = rc.camera.get_color_image()  # Returns a color image in BGR

#-------------------------------
# TODO: CROP IMAGE.  Coordinate is (row, column)
top_left_corner = (0,0)
bottom_right_corner = (480,640)
#-------------------------------

image = rc_utils.crop(image, top_left_corner, bottom_right_corner)
rc.display.show_color_image(image)

**<font style="color:red">Then, copy your HSV bounds from before, create a mask, and demonstrate that the masking process still works on the cropped image.</font>**

In [ ]:
#-------------------------------
# TODO: copy your HSV bounds from before
hsv_lower = (0, 0, 0) # (hue, saturation, value)
hsv_upper = (180, 255, 255) # (hue, saturation, value)

# TODO: include the line of code that uses the HSV
#       bounds to create a mask.


#-------------------------------

# Display the mask
rc.display.show_color_image(mask)

**<font style="color:blue">Finally, run the next code block to see the detected center of the cone in the cropped image. </font>**

In [ ]:
# Find the largest contour
contours = rc_utils.find_contours(mask)
largest_contour = rc_utils.get_largest_contour(contours)

# Draw it on the image
image_copy = np.copy(image)
rc_utils.draw_contour(image_copy, largest_contour)

# Get the center of the contour
center = rc_utils.get_contour_center(largest_contour)

# Draw a circle at the contour center
rc_utils.draw_circle(image_copy, center)
rc.display.show_color_image(image_copy)

## CHECKPOINT:  Show your output to an instructor/TA and be ready to explain what is going on.

## **<font style="color:purple">NEED "Challenge" exercises here for those who finish quickly?</font>**